### Imports

In [1]:
%pip install torch
%pip install torchvision
%pip install phiflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

from phi.torch.flow import *

### Loading data

In [2]:
## CHANGE CODE TO WORK FOR PHIFLOW, COPIED FROM PYTORCH EXAMPLES

class FluidDataset(Dataset):
        """Face Landmarks dataset."""

def __init__(self, csv_file, root_dir, transform=None):
    """
    Args:
        csv_file (string): Path to the csv file with annotations.
        root_dir (string): Directory with all the images.
        transform (callable, optional): Optional transform to be applied
            on a sample.
    """
    self.landmarks_frame = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform

def __len__(self):
    return len(self.landmarks_frame)

def __getitem__(self, idx):
    if torch.is_tensor(idx):
        idx = idx.tolist()

    img_name = os.path.join(self.root_dir,
                            self.landmarks_frame.iloc[idx, 0])
    image = io.imread(img_name)
    landmarks = self.landmarks_frame.iloc[idx, 1:]
    landmarks = np.array([landmarks])
    landmarks = landmarks.astype('float').reshape(-1, 2)
    sample = {'image': image, 'landmarks': landmarks}

    if self.transform:
        sample = self.transform(sample)

    return sample

NameError: name 'Dataset' is not defined

### Basic CNN

In [19]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.drop = nn.Dropout(0.5)
        self.conv1 = nn.Conv2d(4, 8, 2, padding="same") # expected input is 128 x 128 x 4
        self.conv2 = nn.Conv2d(8, 16, 2, padding="same") # size: 64 x 64 x 8
        self.conv3 = nn.Conv2d(16, 32, 2, padding="same") # size: 32 x 32 x 16
        #size: 16 x 16 x 32
        self.deconv3 = nn.ConvTranspose2d(32, 16, 2)
        self.deconv2 = nn.ConvTranspose2d(16, 8, 2)
        self.deconv1 = nn.ConvTranspose2d(8, 4, 2)
        
        

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
#         x = torch.flatten(x, 1) # flatten all dimensions except batch
#         x = self.fc1(x)
        x = F.relu(self.deconv3(x))
        x = np.concatenate([deconv3, conv3])
        x = F.relu(self.deconv2(x))
        x = np.concatenate([deconv2, conv2])
        x = F.relu(self.deconv1(x))
        x = np.concatenate([deconv1, conv1])
        return x

net = Net()

### Loss Function

In [8]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### Training the Network

In [18]:
bounds=Box[0:32, 0:32]
GRAVITY = math.tensor([0, -9.81])
DT = 1
xMax = 128
yMax = 128

water_ref = CenteredGrid(Noise(), extrapolation.ZERO, x=xMax-2, y=yMax-2, bounds=Box[1:xMax-1, 1:yMax-1])
water_ref = CenteredGrid(water_ref, 0, Box[0:xMax, 0:yMax], x=xMax, y=yMax)

velocity_ref = StaggeredGrid(0, extrapolation.ZERO, x=xMax, y=yMax, bounds=bounds)

boundary = CenteredGrid(0, extrapolation.ONE, x=xMax-2, y=yMax-2, bounds=Box[1:xMax-1, 1:yMax-1])
boundary = CenteredGrid(boundary, bounds=Box[0:xMax, 0:yMax], x=xMax, y=yMax)

for epoch in range(10):  # loop over the dataset multiple times

    water = water_ref
    velocity = velocity_ref
    
    velocity += water * GRAVITY * DT @ velocity
    velocity, _ = fluid.make_incompressible(velocity)
    
    running_loss = 0.0
    for i in range(20):
        # get the inputs; data is a list of [inputs, labels]
        density = water.values.numpy(["x","y"])
        boundary_val = boundary.values.numpy(['x', 'y'])
        vel_x = velocity.at_centers().vector['x'].values.numpy(["x","y"])
        vel_y = velocity.at_centers().vector['y'].values.numpy(["x","y"])
        x = np.stack((density, vel_x, vel_y, boundary_val), axis=0)
        x = np.asarray([x])
        x_2 = torch.from_numpy(x)
        
        # run simulator
        water = advect.mac_cormack(water, velocity, dt=DT)
        gravity_force = water * GRAVITY * DT @ velocity
        velocity = advect.semi_lagrangian(velocity, velocity, dt=DT) + gravity_force
        velocity, _ = fluid.make_incompressible(velocity)
        
        density_sim = water.values.numpy(["x", "y"])
        vel_x_sim = velocity.at_centers().vector['x'].values.numpy(["x","y"])
        vel_y_sim = velocity.at_centers().vector['y'].values.numpy(["x","y"])
        y = np.stack((density, vel_x, vel_y, boundary_val), axis=0)
        y = np.asarray([x])
        y_2 = torch.from_numpy(x)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        pred_y = net(x_2)
        loss = criterion(pred_y, y_2)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

NameError: name 'deconv3' is not defined